In [1]:
import cv2
import glfw

import numpy as np
import matplotlib.pyplot as plt

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

from ObjLoader import OBJ
from OpenGLMiscFunctions import loadBackgroundTexture

# Lendo o vídeo

In [2]:
cap = cv2.VideoCapture('entrada.avi')
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if ret == False:
        print('*** Video ended ***')
        break
        
    frames.append(frame)
    
frames = np.array(frames)

# Função show

- A função *show* foi definida para que pudessemos ver as imagens na célula do notebook sem precisarmos abrir a janela extra do OpenCV, facilitando os testes e análises.

In [3]:
def show(img):
    """
        Exibe uma imagem, na própria célula, usando matplotlib
        
        Parâmetros:
        ----------
        img : numpy.ndarray
            Imagem a ser exibida
    """
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(img, cmap=plt.cm.gray)
    ax.set_axis_off()
    plt.show()

# Função homography
- Função criada para estimar a matriz de homografia utilizando a função findHomography com o método RANSAC para filtrar erros grosseiros e a função warpPerspective para aplicar tal matriz na imagem.
- Irá retornar o resultado binarizado para realizarmos posteriormente um template matching mais preciso.

In [4]:
def homography(img, src, target):
    """
        Estima e aplica a matriz de homografia em um conjunto de pontos
        
        Parâmetros
        ----------
        img : numpy.ndarray
            Imagem de onde os pontos foram extraídos
            
        src : numpy.ndarray
            Pontos de origem (da imagem)
            
        target : numpy.ndarray
            Pontos do alvo
            
        Retorno
        -------
        numpy.ndarray
            Retorna aquela parte da imagem binarizada e com a homografia aplicada
    """
    
    # Obtendo os pontos no alvo e estimando a matriz de homografia usando RANSAC
    height, width = target.shape
    dst = np.float32([[0,0], [0, height], [width, height], [width, 0]])
    M = cv2.findHomography(src, dst, cv2.RANSAC)[0]
    
    # Aplicando o warpPerspective e binarizando a imagem
    result = cv2.warpPerspective(img, M, (width, height))
    result[result < 80] = 0
    result[result >= 80] = 255
    
    return result

# Função get_targets
- Função auxiliar usada para ler o arquivo que contêm o alvo, retornando o mesmo rotacionado em 0º, 90º, 180º e 270º respectivamente.

In [5]:
def get_targets(filename):
    """
        Retorna o alvo rotacionado em 0º, 90º, 180º e 270º
        
        Parâmetros
        ----------
        filename : str
            Nome do arquivo, .jpg por exemplo, do alvo
            
        Retorno
        -------
        list
            Lista contendo o alvo rotacionado em 0º, 90º, 180º e 270º (nessa ordem)
    """
    
    # Lendo o alvo, convertendo para escalas de cinza e binarizando
    target = cv2.imread(filename)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    target[target < 25] = 0
    target[target >= 25] = 255
    
    targets_list = []
    
    # Rotacionando o alvo em cada um dos ângulos e salvando em uma lista
    for angle in [0, 90, 180, 270]:
        M = cv2.getRotationMatrix2D((target.shape[1]/2, target.shape[0]/2), angle, 1)
        targets_list.append(cv2.warpAffine(target, M, (target.shape[0], target.shape[1])))
                            
    return targets_list

# Função similarity_func

- Função criada para retornar o quão similar duas imagens são.
- Fizemos um cálculo de diferença média absoluta dos pixels para tal.

In [6]:
def similarity_func(img1, img2):
    """
        Função de similaridade usando a diferença média dos pixels
        
        Parâmetros
        ----------
        img1 : numpy.ndarray
            Imagem 1 a ser comparada
            
        img2 : numpy.ndarray
            Imagem 2 a ser comparada
            
        Retorno
        -------
            Retorna o valor da diferença média dos pixels
    """
    
    return np.sum(np.abs(img1 - img2)) / (img1.size)

# Função template_matching
- Essa função cumpre o papel de: para cada imagem do alvo rotacionado, verificar o quão similar ela é com a imagem obtida no passo da homografia. Salvando o índice da imagem que possui o menor valor retornado pela função similarity_func, em outras palavras, a mais similar.
- Caso essa imagem não possua valor de similaridade menor do que a tolerância definida, não aceitamos e retornamos -1. Caso contrário, retornamos o índice encontrado.

In [7]:
def template_matching(img, targets, sim_func, tol=25):
    """
        Função que realiza o casamento de template
        
        Parâmetros
        ----------
        img : numpy.ndarray
            Imagem que se quer casar
            
        targets : numpy.ndarray list
            Lista de alvos que iremos casar
            
        simFunc : function
            Função de similaridade que aceita dois numpy.ndarray,
            realiza os cálculos e retorna um número
            
        tol : float, opcional
            Tolerância para ser um casamento válido (por padrão é 25)
            
        Retorno
        -------
        int
            Caso o valor de similaridade esteja abaixo da tolerância,
            retornamos o índice do alvo que resultou em um casamento
            bem-sucedido (seguindo a mesma ordem dos ângulos).
            Senão, retornamos -1.
    """
    
    min_similarity = sim_func(img, targets[0])
    min_pos = 0
    
    # Caminhando por cada alvo, comparando o valor de similaridade do atual e 
    # atualizando, caso necessário, as variáveis
    for i in range(1, len(targets)):
        current = sim_func(img, targets[i])
        if current < min_similarity:
            min_similarity = current
            min_pos = i
    
    return min_pos if min_similarity <= tol else -1

# Encontrando os alvos para os 300 primeiros frames

- Nessa célula usaremos a findContours para encontrar os alvos.
- Aliada a isso, iremos usar as funções de homografia e template_matching implementadas no TP1 para verificar se temos um match válido ou não.

In [9]:
targets = get_targets('alvo.jpg')
selected = []
pts = []
pos = []
for frame in frames[:1].copy():
    # Convertendo a imagem para tons de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    
    # Detectando bordas com Canny e contornos
    canny = cv2.Canny(gray, 100, 200)
    contours, _ = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Guardando uma lista de pontos para os possíveis alvos
#     pts = []
    
    for c in contours:
        # Aproximando o contorno via um polígono 
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # Caso esse polígono possua 4 pontos adicionamos ele na nossa lista se a homografia retornar como correto
        if len(approx) == 4:
            homog = homography(np.float32(gray), np.float32(approx), targets[0])
            pos_target = template_matching(homog, targets, similarity_func)
            if  pos_target != -1:
                pos.append(pos_target)
                pts.append(approx)
                
    # Desenhando os contornos
#     for c in pts:
#         cv2.drawContours(frame, c, -1, (0,255,0), 5)

    cv2.circle(frame, tuple(pts[0][0][0]), 5, (0,255,0), -1)
    cv2.circle(frame, tuple(pts[0][1][0]), 5, (255,255,255), -1)
    cv2.circle(frame, tuple(pts[0][2][0]), 5, (0,0,0), -1)
    cv2.circle(frame, tuple(pts[0][3][0]), 5, (100,100,100), -1)
        
    selected.append(frame)

print(pts)
print(pos)
show(selected[0])

# Exibindo os frames

In [10]:
for frame in selected:    
    # Exibindo o frame
    cv2.imshow('Finding targets', frame)
    
    # Caso o usuário aperte 'q', o vídeo termina
    if cv2.waitKey(24) == ord('q'):
        break
        
cv2.destroyAllWindows()

# Renderizando a cena para o primeiro frame

In [ ]:
# Definindo os parâmetros intrínsecos da câmera através da calibração pelo MATLAB
cameraParams = {
    'fc': [536.19341, 536.14756],
    'cc': [317.64968, 233.56773],
    'kc': np.array([0.07459, -0.15101, -0.00544, 0.00111, 0.00000])
}

# Definindo a matriz de parâmetros intrínsecos
intMatrix = np.array([
    [cameraParams['fc'][0], 0.0, cameraParams['cc'][0]],
    [0.0, cameraParams['fc'][1], cameraParams['cc'][1]],
    [0.0, 0.0, 1.0]
])

dst = np.float32([[-1,1,1], [1,1,1], [1,-1,1], [-1,-1,1]])
ret, rvec, tvec = cv2.solvePnP(dst, np.float32(pts[0]), intMatrix, cameraParams['kc'])
rotm = cv2.Rodrigues(rvec)[0]

print('tvec:', tvec)

m = np.array([
    [rotm[0][0], rotm[0][1], rotm[0][2], tvec[0]],
    [rotm[1][0], rotm[1][1], rotm[1][2], -tvec[1]],
    [rotm[2][0], rotm[2][1], rotm[2][2], -tvec[2]],
    [0.0, 0.0, 0.0, 1.0]
])

# m = m * np.array([[ 1.0,  1.0,  1.0,  1.0],
#                   [-1.0, -1.0, -1.0, -1.0],
#                   [-1.0, -1.0, -1.0, -1.0],
#                   [ 1.0,  1.0,  1.0,  1.0]])

m = np.transpose(m)

# Definindo a largura e altura da nossa janela
width, height = 640, 480

# Definindo a função que desenha o background com textura
def showBackground():
    glDepthMask(GL_FALSE)
    
    # Definindo a projeção como ortográfica
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    
    # Habilitando a textura
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, textureId)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    # Desenhando um quadrilátero do tamanho da tela com a textura
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0); glVertex2f(0, 0)
    glTexCoord2f(1, 0); glVertex2f(width, 0)
    glTexCoord2f(1, 1); glVertex2f(width, height)
    glTexCoord2f(0, 1); glVertex2f(0, height)
    glEnd()
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)
    
    # Desabilitando a textura e chamando glFlush
    glBindTexture(GL_TEXTURE_2D, 0)
    glDepthMask(GL_TRUE)
    glFlush()

# Definindo a função de desenhar
def draw():
    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # Desenhando o fundo
    showBackground()
    
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glPushMatrix()
    glLoadMatrixf(m)
    glCallList(obj.gl_list)
    glPopMatrix()
    
    # Trocando de buffer para exibir na tela
    glutSwapBuffers()
    
# Iniciando o GLUT
glutInit()

# Iniciando a posição da janela, dimensões e criando a memsa
glutInitWindowPosition(0, 0)
glutInitWindowSize(width, height)
glutCreateWindow('TP2')

# Lendo a textura do fundo
textureId = loadBackgroundTexture(frames[0])

# Habilitando o uso de texturas
glEnable(GL_TEXTURE_2D)

glEnable(GL_DEPTH_TEST)

obj = OBJ('Pikachu.obj', swapyz=True)

glMatrixMode(GL_PROJECTION)
glLoadIdentity()
fx = cameraParams['fc'][0]
fy = cameraParams['fc'][1]
fovy = 2 * np.arctan(0.5 * height / fy)*180 / np.pi
aspect = (width*fy) / (height*fx)
gluPerspective(fovy, aspect, 0.1, 100.0)

# Definindo a cor de limpeza do fundo
glClearColor(0.2, 0.2, 0.2, 0.0)

# Iniciando o loop
glutDisplayFunc(draw)
# glutIdleFunc(draw)
glutMainLoop()

In [ ]:
# # Definindo os parâmetros intrínsecos da câmera através da calibração pelo MATLAB
# cameraParams = {
#     'fc': [536.19341, 536.14756],
#     'cc': [317.64968, 233.56773],
#     'kc': np.array([0.07459, -0.15101, -0.00544, 0.00111, 0.00000])
# }

# # Definindo a matriz de parâmetros intrínsecos
# intMatrix = np.array([
#     [cameraParams['fc'][0], 0.0, cameraParams['cc'][0]],
#     [0.0, cameraParams['fc'][1], cameraParams['cc'][1]],
#     [0.0, 0.0, 1.0]
# ])

# ret, rvec, tvec = cv2.solvePnP(np.float32([[0,0,0], [0,310,0], [310,310,0], [310,0,0]]), np.float32(pts[0]), intMatrix, cameraParams['kc'])
# rotm = cv2.Rodrigues(rvec)[0]

# m = np.array([
#     [rotm[0][0], rotm[0][1], rotm[0][2], 0.0],
#     [rotm[1][0], rotm[1][1], rotm[1][2], 0.0],
#     [rotm[2][0], rotm[2][1], rotm[2][2], 0.0],
#     [0.0, 0.0, 0.0, 1.0]
# ])

# m = m * np.array([[ 1.0,  1.0,  1.0,  1.0],
#                   [-1.0, -1.0, -1.0, -1.0],
#                   [-1.0, -1.0, -1.0, -1.0],
#                   [ 1.0,  1.0,  1.0,  1.0]])

# m = np.transpose(m)

# # Definindo a largura e altura da nossa janela
# width, height = 640, 480

# if not glfw.init():
#     print('*** OPS ***')
    
# window = glfw.create_window(640, 480, 'TESTE', None, None)
# glfw.make_context_current(window)

# fx = cameraParams['fc'][0]
# fy = cameraParams['fc'][1]
# fovy = 2 * np.arctan(0.5 * height / fy)*180 / np.pi
# aspect = (width*fy) / (height*fx)
# gluPerspective(fovy, aspect, 0.1, 100.0)

# obj = OBJ('Pikachu.obj', swapyz=True)

# glClearColor(0.2, 0.2, 0.2, 0.0)

# while not glfw.window_should_close(window):
#     glfw.poll_events()
    
#     # Limpando os buffers
#     glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
#     # Desenhando o fundo
# #     showBackground()
    
#     glMatrixMode(GL_MODELVIEW)
#     glLoadIdentity()
#     glPushMatrix()
#     glLoadMatrixd(m)
#     glCallList(obj.gl_list)
#     glPopMatrix()
    
#     glfw.swap_buffers(window)